In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from scipy.stats import linregress
import warnings
import os
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from tabpfn import TabPFNRegressor
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 匹配参考图的样式配置
plt.rcParams['figure.dpi'] = 300
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linestyle'] = '-'

# ============ 1. 加载数据 ============
print("="*60)
print("加载数据")
print("="*60)

# 检查模型文件
LOCAL_MODEL_PATH = "tabpfn-v2-regressor.ckpt"
tabpfn_available = os.path.exists(LOCAL_MODEL_PATH)

# 加载数据（支持多种格式）
data_files = ["dataset/tabpfn_dataset_1000.xlsx"]
df = None
for file in data_files:
    try:
        if file.endswith('.xlsx'):
            df = pd.read_excel(file)
        else:
            df = pd.read_csv(file)
        print(f"✓ 成功加载数据: {file}")
        break
    except FileNotFoundError:
        continue

if df is None:
    print("错误：找不到数据文件")
    print("请确保以下文件之一存在:", data_files)
    exit()

print(f"数据形状: {df.shape}")
print(f"列名: {list(df.columns)}")

# ============ 2. 多重共线性检验 (VIF分析) ============
print("\n" + "="*60)
print("多重共线性检验 (VIF分析)")
print("="*60)

def calculate_vif(X_data, threshold=5):
    """计算VIF值并识别高度共线性特征"""
    print("计算方差膨胀因子(VIF)...")
    
    # 确保是DataFrame格式
    if not isinstance(X_data, pd.DataFrame):
        X_data = pd.DataFrame(X_data)
    
    # 添加常数项
    X_with_const = add_constant(X_data)
    
    # 计算VIF
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_with_const.columns
    vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) 
                      for i in range(X_with_const.shape[1])]
    
    # 移除常数项
    vif_data = vif_data[vif_data["Feature"] != "const"]
    
    # 分类
    vif_data["Collinearity_Level"] = vif_data["VIF"].apply(
        lambda x: "Severe (VIF ≥ 10)" if x >= 10 else 
                  "Moderate (5 ≤ VIF < 10)" if x >= 5 else 
                  "Acceptable (1 < VIF < 5)" if x > 1 else "Low (VIF ≤ 1)"
    )
    
    vif_data["Collinearity_Concern"] = vif_data["VIF"].apply(
        lambda x: "HIGH" if x >= 10 else "MODERATE" if x >= 5 else "LOW"
    )
    
    return vif_data.sort_values("VIF", ascending=False)

# 准备数据
target_col = 'engagement_score'
if target_col not in df.columns:
    print(f"警告：找不到 '{target_col}' 列，使用第一列作为目标变量")
    target_col = df.columns[0]

X = df.drop(columns=[target_col])
y = df[target_col]

# 只选择数值型特征进行VIF分析
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
if len(numeric_cols) > 0:
    X_numeric = X[numeric_cols]
    vif_results = calculate_vif(X_numeric)
    
    print("\n📊 VIF分析结果:")
    print("-" * 80)
    print(vif_results.to_string(index=False))
    
    # 统计总结
    high_vif = vif_results[vif_results["VIF"] >= 10]
    moderate_vif = vif_results[(vif_results["VIF"] >= 5) & (vif_results["VIF"] < 10)]
    
    print(f"\n📈 多重共线性总结:")
    print(f"  严重共线性 (VIF ≥ 10): {len(high_vif)} 个特征")
    print(f"  中度共线性 (5 ≤ VIF < 10): {len(moderate_vif)} 个特征")
    print(f"  可接受共线性 (VIF < 5): {len(vif_results) - len(high_vif) - len(moderate_vif)} 个特征")
    
    if len(high_vif) > 0:
        print(f"\n⚠️  建议处理以下高VIF特征:")
        for _, row in high_vif.iterrows():
            print(f"  {row['Feature']}: VIF = {row['VIF']:.2f}")
    
    # 可视化VIF结果
    plt.figure(figsize=(12, 6))
    colors = ['red' if v >= 10 else 'orange' if v >= 5 else 'green' for v in vif_results["VIF"]]
    bars = plt.barh(range(len(vif_results)), vif_results["VIF"], color=colors)
    plt.axvline(x=5, color='orange', linestyle='--', alpha=0.7, label='VIF = 5 (阈值)')
    plt.axvline(x=10, color='red', linestyle='--', alpha=0.7, label='VIF = 10 (严重)')
    plt.xlabel('Variance Inflation Factor (VIF)')
    plt.title('多重共线性检验 (VIF分析)')
    plt.yticks(range(len(vif_results)), vif_results["Feature"])
    plt.legend()
    plt.tight_layout()
    plt.savefig('vif_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ VIF可视化已保存为 'vif_analysis.png'")
    
    # 保存VIF结果
    vif_results.to_excel('vif_analysis_results.xlsx', index=False)
    print("✓ VIF分析结果已保存到 'vif_analysis_results.xlsx'")
else:
    print("⚠️  没有数值型特征，跳过VIF分析")

# ============ 3. 划分数据集 ============
print("\n" + "="*60)
print("数据集划分")
print("="*60)

# 划分数据集（8:2，保留20%作为最终测试集）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"训练集: {X_train.shape}, 测试集: {X_test.shape}")

# ============ 4. 数据标准化 ============
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ============ 5. 初始化模型 ============
print("\n" + "="*60)
print("初始化模型")
print("="*60)

# 首先定义TabPFN包装器类
class TabPFNWrapper:
    def __init__(self):
        self.model = None
        self.is_fitted = False
        
    def fit(self, X, y):
        try:
            from tabpfn import TabPFNRegressor
            if tabpfn_available:
                self.model = TabPFNRegressor(device='cpu', model_path=LOCAL_MODEL_PATH)
                self.model.fit(X.values if hasattr(X, 'values') else X, 
                              y.values if hasattr(y, 'values') else y)
                self.is_fitted = True
            else:
                raise ImportError("TabPFN模型文件未找到")
        except ImportError as e:
            print(f"TabPFN不可用: {e}")
            raise
        return self
    
    def predict(self, X):
        if self.model is None or not self.is_fitted:
            raise ValueError("模型未训练")
        return self.model.predict(X.values if hasattr(X, 'values') else X)
    
    def get_params(self, deep=True):
        return {}
    
    def set_params(self, **params):
        return self

# 定义所有模型
models = {
    'SVR': (SVR(), {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}),
    'KNN': (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7, 9, 11, 15, 20, 40]}),
    'GBM': (GradientBoostingRegressor(random_state=42), {
        'n_estimators': [50, 100, 200, 300], 
        'learning_rate': [0.01, 0.1, 0.5, 1, 5], 
        'max_depth': [5, 10, 15, 20, 40, 50]
    }),
    'RandomForest': (RandomForestRegressor(random_state=42), {
        'n_estimators': [50, 100, 200, 300], 
        'max_depth': [5, 10, 15, 20], 
        'min_samples_split': [4, 8, 12, 16, 20]
    }),
    'Ridge': (Ridge(random_state=42), {'alpha': [0.1, 1, 10, 100]}),
    'XGBoost': (xgb.XGBRegressor(random_state=42, verbosity=0), {
        'n_estimators': [50, 100, 200], 
        'learning_rate': [0.01, 0.05, 0.1], 
        'max_depth': [3, 6, 10], 
        'subsample': [0.8, 1.0]
    }),
    'LightGBM': (lgb.LGBMRegressor(random_state=42, verbose=-1), {
        'n_estimators': [50, 100, 200], 
        'learning_rate': [0.01, 0.05, 0.1], 
        'max_depth': [3, 6, 10], 
        'num_leaves': [5, 10, 15, 20, 30]
    })
}

# 添加TabPFN（如果可用）
if tabpfn_available:
    try:
        models['TabPFN'] = (TabPFNWrapper(), {})
        print("✓ TabPFN已添加到模型列表")
    except Exception as e:
        print(f"⚠️  TabPFN初始化失败: {e}")
else:
    print("⚠️  TabPFN模型文件未找到，跳过TabPFN")

print(f"\n总共比较 {len(models)} 个模型: {list(models.keys())}")

# ============ 6. 训练和评估模型 ============
print("\n" + "="*60)
print("训练和评估模型 (10折交叉验证)")
print("="*60)

results = {}
model_predictions = {}

for name, (model, param_grid) in models.items():
    print(f"\n--- 训练 {name} ---")
    
    try:
        # 区分TabPFN和其他模型
        is_tabpfn = name == 'TabPFN'
        
        if is_tabpfn:
            # TabPFN：使用原始数据，无网格搜索
            best_model = model
            best_model.fit(X_train, y_train)
            best_params = "TabPFN (预训练模型)"
            X_eval_train = X_train
            X_eval_test = X_test
            
            # 对于TabPFN，我们需要特殊的交叉验证
            cv_rmse_scores = []
            for i in range(10):
                # 使用不同的随机种子创建训练/验证分割
                X_train_cv, X_val_cv, y_train_cv, y_val_cv = train_test_split(
                    X_train, y_train, test_size=0.1, random_state=i
                )
                
                # 训练TabPFN
                temp_model = TabPFNWrapper()
                temp_model.fit(X_train_cv, y_train_cv)
                
                # 预测并计算RMSE
                y_pred_cv = temp_model.predict(X_val_cv)
                rmse_cv = np.sqrt(np.mean((y_val_cv - y_pred_cv) ** 2))
                cv_rmse_scores.append(rmse_cv)
            
            cv_rmse_scores = np.array(cv_rmse_scores)
            
        else:
            # 其他模型：网格搜索 + 标准化数据
            grid_search = GridSearchCV(
                estimator=model,
                param_grid=param_grid,
                cv=10,
                scoring='neg_mean_squared_error',
                n_jobs=-1,
                verbose=0
            )
            
            grid_search.fit(X_train_scaled, y_train)
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            X_eval_train = X_train_scaled
            X_eval_test = X_test_scaled
            
            # 交叉验证分数
            cv_scores = cross_val_score(
                best_model, X_eval_train, y_train,
                cv=10, scoring='neg_mean_squared_error', n_jobs=-1
            )
            cv_rmse_scores = np.sqrt(-cv_scores)
        
        # 测试集评估
        y_pred = best_model.predict(X_eval_test)
        
        # 计算性能指标
        mse = np.mean((y_test - y_pred) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_test - y_pred))
        r2 = 1 - np.sum((y_test - y_pred) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2)
        
        # ====== 新增：计算MAPE ======
        # 避免除零错误
        mask = y_test != 0  # 筛选出y_test非零的样本
        if np.any(mask):
            # 只计算非零样本的MAPE
            mape = np.mean(np.abs((y_test[mask] - y_pred[mask]) / y_test[mask])) * 100
        else:
            # 若所有y_test都是0，MAPE无意义，设为0或NaN
            mape = np.nan  # 或 0.0，根据业务需求选择
        
        # 保存结果
        results[name] = {
            'model': best_model,
            'rmse': rmse,
            'r2': r2,
            'mae': mae,
            'mape': mape,  # 新增
            'cv_rmse_scores': cv_rmse_scores,
            'best_params': best_params,
            'y_pred': y_pred,
            'y_test': y_test.values
        }
        
        model_predictions[name] = y_pred
        
        print(f"  测试集 RMSE: {rmse:.4f}")
        print(f"  测试集 R²: {r2:.4f}")
        print(f"  测试集 MAE: {mae:.4f}")
        print(f"  测试集 MAPE: {mape:.2f}%")  # 新增
        print(f"  交叉验证 RMSE: {np.mean(cv_rmse_scores):.4f} ± {np.std(cv_rmse_scores):.4f}")
        
        if not is_tabpfn:
            print(f"  最佳参数: {best_params}")
        
    except Exception as e:
        print(f"  ✗ 训练失败: {e}")
        import traceback
        traceback.print_exc()

# ============ 7. 置信区间计算 ============
print("\n" + "="*60)
print("置信区间计算 (95% CI)")
print("="*60)

def calculate_confidence_interval(data, confidence=0.95):
    """计算置信区间"""
    n = len(data)
    if n < 2:
        return np.mean(data), np.mean(data), np.mean(data)
    
    mean = np.mean(data)
    sem = stats.sem(data)  # 标准误
    t_val = stats.t.ppf((1 + confidence) / 2, n - 1)
    ci = t_val * sem
    
    return mean, mean - ci, mean + ci

# 为每个模型计算置信区间
performance_summary = []

for name, res in results.items():
    # 计算交叉验证RMSE的置信区间
    cv_mean, ci_lower, ci_upper = calculate_confidence_interval(res['cv_rmse_scores'])
    
    # 计算测试集指标的置信区间（使用Bootstrap）
    n_bootstrap = 1000
    bootstrap_rmse = []
    bootstrap_r2 = []
    bootstrap_mae = []
    bootstrap_mape = []  # 新增
    
    n_samples = len(y_test)
    for _ in range(n_bootstrap):
        # Bootstrap重采样
        indices = np.random.choice(n_samples, n_samples, replace=True)
        y_test_boot = y_test.values[indices]
        y_pred_boot = res['y_pred'][indices]
        
        # 避免除零（针对MAPE计算）
        y_test_boot_nonzero = y_test_boot.copy()
        y_test_boot_nonzero[y_test_boot_nonzero == 0] = 1e-10
        
        # 计算指标
        bootstrap_rmse.append(np.sqrt(np.mean((y_test_boot - y_pred_boot) ** 2)))
        bootstrap_r2.append(1 - np.sum((y_test_boot - y_pred_boot) ** 2) / 
                           np.sum((y_test_boot - np.mean(y_test_boot)) ** 2))
        bootstrap_mae.append(np.mean(np.abs(y_test_boot - y_pred_boot)))
        bootstrap_mape.append(np.mean(np.abs((y_test_boot - y_pred_boot) / y_test_boot_nonzero)) * 100)  # 新增
    
    # Bootstrap置信区间
    rmse_ci_lower, rmse_ci_upper = np.percentile(bootstrap_rmse, [2.5, 97.5])
    r2_ci_lower, r2_ci_upper = np.percentile(bootstrap_r2, [2.5, 97.5])
    mae_ci_lower, mae_ci_upper = np.percentile(bootstrap_mae, [2.5, 97.5])
    mape_ci_lower, mape_ci_upper = np.percentile(bootstrap_mape, [2.5, 97.5])  # 新增
    
    performance_summary.append({
        'Model': name,
        # 测试集指标 + Bootstrap CI
        'Test_RMSE': f"{res['rmse']:.4f} [{rmse_ci_lower:.4f}, {rmse_ci_upper:.4f}]",
        'Test_MAE': f"{res['mae']:.4f} [{mae_ci_lower:.4f}, {mae_ci_upper:.4f}]",
        'Test_MAPE': f"{res['mape']:.1f}% [{mape_ci_lower:.1f}%, {mape_ci_upper:.1f}%]",  # 新增
        'Test_R2': f"{res['r2']:.4f} [{r2_ci_lower:.4f}, {r2_ci_upper:.4f}]",
        # 交叉验证指标 + 理论CI
        'CV_RMSE_mean': cv_mean,
        'CV_RMSE_std': np.std(res['cv_rmse_scores']),
        'CV_RMSE_CI_lower': ci_lower,
        'CV_RMSE_CI_upper': ci_upper,
        'CV_RMSE_CI_width': ci_upper - ci_lower,
        # 原始数值（用于排序）
        'rmse_value': res['rmse'],
        'mae_value': res['mae'],
        'mape_value': res['mape'],  # 新增
        'r2_value': res['r2']
    })

# 创建DataFrame并排序
summary_df = pd.DataFrame(performance_summary)
summary_df_sorted = summary_df.sort_values('rmse_value')

print("\n📊 模型性能总结（按测试集RMSE排序）:")
print("-" * 140)
print(f"{'模型':<12} {'测试集RMSE (95% CI)':<30} {'测试集MAE (95% CI)':<30} {'测试集MAPE (95% CI)':<25} {'测试集R² (95% CI)':<30}")
print("-" * 140)

for _, row in summary_df_sorted.iterrows():
    print(f"{row['Model']:<12} {row['Test_RMSE']:<30} {row['Test_MAE']:<30} "
          f"{row['Test_MAPE']:<25} {row['Test_R2']:<30}")

# ============ 8. 统计显著性检验 ============
print("\n" + "="*60)
print("统计显著性检验")
print("="*60)

if len(results) >= 2:
    # 获取所有模型的交叉验证分数
    model_names = list(results.keys())
    cv_scores_dict = {name: results[name]['cv_rmse_scores'] for name in model_names}
    
    # 识别最佳模型
    best_model_name = summary_df_sorted.iloc[0]['Model']
    print(f"\n配对t检验：所有模型 vs 最佳模型 ({best_model_name})")
    print("=" * 100)
    print("注意：")
    print("1. 差异 = 对比模型RMSE - 最佳模型RMSE")
    print("2. 正值表示最佳模型更好（RMSE更小）")
    print("3. t统计量检验差异是否显著大于0")
    print("=" * 100)
    
    t_test_results = []
    for model_name in model_names:
        if model_name != best_model_name:
            # 获取分数
            scores_best = cv_scores_dict[best_model_name]
            scores_other = cv_scores_dict[model_name]
            
            # 确保长度相同
            min_len = min(len(scores_best), len(scores_other))
            if min_len >= 2:
                scores_best_adj = scores_best[:min_len]
                scores_other_adj = scores_other[:min_len]
                
                # ========== 关键修正：正确的比较方向 ==========
                # 计算差异：对比模型 - 最佳模型
                # 如果最佳模型更好（RMSE更小），差异应该为正数
                differences = scores_other_adj - scores_best_adj
                
                # 配对t检验：检验差异是否显著大于0（单侧检验）
                # 使用stats.t.cdf进行单侧检验
                t_stat, p_value_two_sided = stats.ttest_rel(differences, 0)
                
                # 转换为单侧p值（检验差异是否>0）
                if t_stat > 0:
                    # t值为正，检验差异是否显著大于0
                    p_value_one_sided = p_value_two_sided / 2
                else:
                    # t值为负，差异小于0，对当前假设不利
                    p_value_one_sided = 1 - p_value_two_sided / 2
                
                # 均值差（对比模型 - 最佳模型）
                mean_diff = np.mean(differences)
                
                # 效应量
                pooled_std = np.sqrt((np.var(scores_best_adj) + np.var(scores_other_adj)) / 2)
                cohens_d = mean_diff / pooled_std if pooled_std != 0 else 0
                
                # ========== 验证打印 ==========
                print(f"\n📊 验证 {best_model_name} vs {model_name}:")
                print(f"  {best_model_name}平均RMSE: {np.mean(scores_best_adj):.4f}")
                print(f"  {model_name}平均RMSE: {np.mean(scores_other_adj):.4f}")
                print(f"  差异({model_name} - {best_model_name}): {mean_diff:.4f}")
                print(f"  t统计量: {t_stat:.3f}")
                print(f"  双侧p值: {p_value_two_sided:.4f}")
                print(f"  单侧p值(差异>0): {p_value_one_sided:.4f}")
                
                # 效应量解释
                if abs(cohens_d) >= 0.8:
                    effect_magnitude = "大"
                elif abs(cohens_d) >= 0.5:
                    effect_magnitude = "中"
                elif abs(cohens_d) >= 0.2:
                    effect_magnitude = "小"
                else:
                    effect_magnitude = "可忽略"
                
                # 判断显著性（单侧检验）
                sig_005 = p_value_one_sided < 0.05 and mean_diff > 0
                sig_001 = p_value_one_sided < 0.01 and mean_diff > 0
                sig_0001 = p_value_one_sided < 0.001 and mean_diff > 0
                
                # 决定使用哪个t值：如果mean_diff为正，但t_stat为负，取绝对值
                # 这通常发生在计算顺序问题时
                report_t_stat = t_stat
                if mean_diff > 0 and t_stat < 0:
                    print(f"  ⚠️  注意：均值差为正但t值为负，使用|t|值")
                    report_t_stat = abs(t_stat)
                
                t_test_results.append({
                    'Comparison': f"{best_model_name} vs {model_name}",
                    'Best_Mean_RMSE': np.mean(scores_best_adj),
                    'Other_Mean_RMSE': np.mean(scores_other_adj),
                    'Mean_Difference': mean_diff,  # 对比模型 - 最佳模型
                    't_statistic': report_t_stat,  # 修正后的t值
                    'p_value_two_sided': p_value_two_sided,
                    'p_value_one_sided': p_value_one_sided,
                    'cohens_d': cohens_d,
                    'effect_magnitude': effect_magnitude,
                    'significant_0.05': sig_005,
                    'significant_0.01': sig_001,
                    'significant_0.001': sig_0001,
                    'sig_stars': "***" if sig_0001 else "**" if sig_001 else "*" if sig_005 else ""
                })
    
    # 创建t检验结果DataFrame
    if t_test_results:
        t_test_df = pd.DataFrame(t_test_results)
        
        print(f"\n" + "="*100)
        print(f"配对t检验最终结果（使用单侧检验）:")
        print("="*100)
        print(f"{'比较':<25} {'均值差':<12} {'t统计量':<10} {'p值(单侧)':<12} {'Cohen\'s d':<10} {'效应量':<10} {'显著性':<10}")
        print("-" * 100)
        
        for _, row in t_test_df.iterrows():
            print(f"{row['Comparison']:<25} {row['Mean_Difference']:>11.4f} {row['t_statistic']:>9.3f} "
                  f"{row['p_value_one_sided']:>11.4f} {row['cohens_d']:>9.3f} "
                  f"{row['effect_magnitude']:<10} {row['sig_stars']:<10}")
        
        # 添加解释说明
        print("\n" + "="*100)
        print("结果解释说明:")
        print("="*100)
        print("1. 均值差 = 对比模型RMSE - 最佳模型RMSE")
        print("2. 正值表示最佳模型的RMSE更小（性能更好）")
        print("3. t统计量检验均值差是否显著大于0")
        print("4. 使用单侧p值，检验最佳模型是否显著优于对比模型")
        print("5. 效应量解释：")
        print("   - d ≥ 0.8: 大效应")
        print("   - 0.5 ≤ d < 0.8: 中等效应")
        print("   - 0.2 ≤ d < 0.5: 小效应")
        print("   - d < 0.2: 可忽略效应")
        print("="*100)

# ============ 9. 可视化结果（完全匹配参考图风格） ============
print("\n" + "="*60)
print("结果可视化（匹配参考图风格）")
print("="*60)

# 9.1 模型性能对比图（带置信区间）
plt.figure(figsize=(14, 8))

# 子图1：测试集RMSE对比
plt.subplot(2, 2, 1)
model_names_plot = summary_df_sorted['Model'].tolist()
rmse_values = summary_df_sorted['rmse_value'].tolist()

# 提取置信区间
rmse_ci_lower = []
rmse_ci_upper = []
for rmse_str in summary_df_sorted['Test_RMSE']:
    ci_parts = rmse_str.split('[')[1].split(']')[0].split(', ')
    rmse_ci_lower.append(float(ci_parts[0]))
    rmse_ci_upper.append(float(ci_parts[1]))

ci_errors = [rmse_values[i] - rmse_ci_lower[i] for i in range(len(rmse_values))]
ci_errors_upper = [rmse_ci_upper[i] - rmse_values[i] for i in range(len(rmse_values))]

bars = plt.bar(range(len(model_names_plot)), rmse_values, 
               yerr=[ci_errors, ci_errors_upper], capsize=5, alpha=0.7)
plt.xlabel('模型')
plt.ylabel('测试集RMSE')
plt.title('模型性能对比 (RMSE, 95%置信区间)')
plt.xticks(range(len(model_names_plot)), model_names_plot, rotation=45, ha='right')
plt.grid(True, alpha=0.3)

# 为最佳模型着色
best_idx = rmse_values.index(min(rmse_values))
bars[best_idx].set_color('green')
bars[best_idx].set_alpha(0.9)

# 子图2：MAE对比
plt.subplot(2, 2, 2)
mae_values = summary_df_sorted['mae_value'].tolist()

# 提取MAE置信区间
mae_ci_lower = []
mae_ci_upper = []
for mae_str in summary_df_sorted['Test_MAE']:
    ci_parts = mae_str.split('[')[1].split(']')[0].split(', ')
    mae_ci_lower.append(float(ci_parts[0]))
    mae_ci_upper.append(float(ci_parts[1]))

mae_ci_errors = [mae_values[i] - mae_ci_lower[i] for i in range(len(mae_values))]
mae_ci_errors_upper = [mae_ci_upper[i] - mae_values[i] for i in range(len(mae_values))]

bars2 = plt.bar(range(len(model_names_plot)), mae_values, 
                yerr=[mae_ci_errors, mae_ci_errors_upper], capsize=5, alpha=0.7, color='orange')
plt.xlabel('模型')
plt.ylabel('测试集MAE')
plt.title('模型性能对比 (MAE, 95%置信区间)')
plt.xticks(range(len(model_names_plot)), model_names_plot, rotation=45, ha='right')
plt.grid(True, alpha=0.3)
bars2[best_idx].set_color('green')
bars2[best_idx].set_alpha(0.9)

# 子图3：MAPE对比
plt.subplot(2, 2, 3)
mape_values = summary_df_sorted['mape_value'].tolist()

# 提取MAPE置信区间
mape_ci_lower = []
mape_ci_upper = []
for mape_str in summary_df_sorted['Test_MAPE']:
    ci_parts = mape_str.split('[')[1].split('%]')[0].split('%, ')
    mape_ci_lower.append(float(ci_parts[0]))
    mape_ci_upper.append(float(ci_parts[1]))

mape_ci_errors = [mape_values[i] - mape_ci_lower[i] for i in range(len(mape_values))]
mape_ci_errors_upper = [mape_ci_upper[i] - mape_values[i] for i in range(len(mape_values))]

bars3 = plt.bar(range(len(model_names_plot)), mape_values, 
                yerr=[mape_ci_errors, mape_ci_errors_upper], capsize=5, alpha=0.7, color='purple')
plt.xlabel('模型')
plt.ylabel('测试集MAPE (%)')
plt.title('模型性能对比 (MAPE, 95%置信区间)')
plt.xticks(range(len(model_names_plot)), model_names_plot, rotation=45, ha='right')
plt.grid(True, alpha=0.3)
bars3[best_idx].set_color('green')
bars3[best_idx].set_alpha(0.9)

# 子图4：最佳模型散点图（匹配参考图风格）
plt.subplot(2, 2, 4)
best_model_name = summary_df_sorted.iloc[0]['Model']
y_pred_best = results[best_model_name]['y_pred']
y_test_best = results[best_model_name]['y_test']

# 计算拟合线
slope, intercept, r_value, p_value, std_err = linregress(y_test_best, y_pred_best)
fit_line = intercept + slope * y_test_best

# ========== 核心修改：全局固定宽度置信区间（匹配参考图） ==========
# 计算全局误差标准差
errors = y_pred_best - y_test_best
std_error_global = np.std(errors)
ci_95_global = 1.96 * std_error_global  # 95%置信区间宽度

# 绘制散点图（Jet配色+色条）
sc = plt.scatter(y_test_best, y_pred_best, c=y_pred_best, cmap='jet', alpha=0.8, s=30)
# 理想线（紫色虚线，贯穿整个0-1区间）
plt.plot([0, 1], [0, 1], 'purple', linestyle='--', linewidth=2, label='理想线 (y=x)')
# 拟合线（红色实线）
plt.plot(y_test_best, fit_line, 'red', linewidth=2, 
         label=f'拟合线: y={intercept:.4f}+{slope:.4f}x')

# ========== 全局固定宽度置信区间（匹配参考图） ==========
plt.fill_between(y_test_best, 
                 fit_line - ci_95_global, 
                 fit_line + ci_95_global, 
                 color='pink', alpha=0.3, label='95%置信区间')

# 添加指标标注（匹配参考图位置和格式）
r2 = results[best_model_name]['r2']
rmse = results[best_model_name]['rmse']
mae = results[best_model_name]['mae']
mape = results[best_model_name]['mape']  # 新增
plt.text(0.05, 0.95, 
         f'$R^2={r2:.3f}$\nRMSE={rmse:.3f}\nMAE={mae:.3f}\nMAPE={mape:.1f}%',  # 新增MAPE
         transform=plt.gca().transAxes, 
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
         fontsize=10)

# 添加色条
cbar = plt.colorbar(sc)
cbar.set_label('预测值', fontsize=10)

# 设置坐标轴范围（0-1）
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('观测值', fontsize=10)
plt.ylabel('预测值', fontsize=10)
plt.title(f'最佳模型: {best_model_name}', fontsize=12)
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_comparison_with_ci.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ 模型对比图已保存为 'model_comparison_with_ci.png'")

# ============ 生成所有模型的散点图（完全匹配参考图风格） ============
for model_name in results.keys():
    y_pred = results[model_name]['y_pred']
    y_test_vals = results[model_name]['y_test']
    r2 = results[model_name]['r2']
    rmse = results[model_name]['rmse']
    mae = results[model_name]['mae']
    mape = results[model_name]['mape']  # 新增
    
    # 计算拟合线
    slope, intercept, r_value, p_value, std_err = linregress(y_test_vals, y_pred)
    fit_line = intercept + slope * y_test_vals
    
    # ========== 核心修改：全局固定宽度置信区间（匹配参考图） ==========
    # 计算全局误差标准差
    errors = y_pred - y_test_vals
    std_error_global = np.std(errors)
    ci_95_global = 1.96 * std_error_global  # 95%置信区间宽度
    
    # 创建图表（匹配参考图尺寸）
    plt.figure(figsize=(8, 7))
    
    # 绘制散点图（Jet配色+色条）
    sc = plt.scatter(y_test_vals, y_pred, c=y_pred, cmap='jet', alpha=0.8, s=30)
    
    # 理想线（紫色虚线，贯穿整个0-1区间）
    plt.plot([0, 1], [0, 1], 'purple', linestyle='--', linewidth=2, label='理想线 (y=x)')
    
    # 拟合线（红色实线）
    plt.plot(y_test_vals, fit_line, 'red', linewidth=2, 
             label=f'拟合线: y={intercept:.4f}+{slope:.4f}x')
    
    # ========== 全局固定宽度置信区间（匹配参考图） ==========
    plt.fill_between(y_test_vals, 
                     fit_line - ci_95_global, 
                     fit_line + ci_95_global, 
                     color='pink', alpha=0.3, label='95%置信区间')
    
    # 指标标注（匹配参考图格式）
    plt.text(0.05, 0.95, 
             f'$R^2={r2:.3f}$\nRMSE={rmse:.3f}\nMAE={mae:.3f}\nMAPE={mape:.1f}%',  # 新增MAPE
             transform=plt.gca().transAxes, 
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
             fontsize=10)
    
    # 色条
    cbar = plt.colorbar(sc)
    cbar.set_label('预测值', fontsize=10)
    
    # 设置坐标轴范围（0-1），匹配参考图
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    
    # 标签和标题
    plt.xlabel('观测值', fontsize=12)
    plt.ylabel('预测值', fontsize=12)
    plt.title(f'{model_name} 模型 - 预测值 vs 观测值', fontsize=14)
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # 保存（高分辨率）
    plt.tight_layout()
    plt.savefig(f'{model_name}_predictions_scatter.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ {model_name} 散点图已保存为 '{model_name}_predictions_scatter.png'")

# ============ 生成所有模型汇总散点图（匹配风格） ============
model_count = len(results)
n_rows = (model_count + 1) // 2
n_cols = 2 if model_count > 1 else 1

plt.figure(figsize=(14, 6 * n_rows))
for idx, (model_name, res) in enumerate(results.items()):
    y_pred = res['y_pred']
    y_test_vals = res['y_test']
    r2 = res['r2']
    rmse = res['rmse']
    mape = res['mape']  # 新增
    
    # 计算拟合线
    slope, intercept, r_value, p_value, std_err = linregress(y_test_vals, y_pred)
    fit_line = intercept + slope * y_test_vals
    
    # ========== 全局固定宽度置信区间 ==========
    errors = y_pred - y_test_vals
    std_error_global = np.std(errors)
    ci_95_global = 1.96 * std_error_global
    
    # 创建子图
    plt.subplot(n_rows, n_cols, idx + 1)
    
    # 绘制散点图
    sc = plt.scatter(y_test_vals, y_pred, c=y_pred, cmap='jet', alpha=0.8, s=20)
    # 理想线（贯穿0-1）
    plt.plot([0, 1], [0, 1], 'purple', linestyle='--', linewidth=2)
    # 拟合线
    plt.plot(y_test_vals, fit_line, 'red', linewidth=2)
    # 全局置信区间
    plt.fill_between(y_test_vals, fit_line - ci_95_global, fit_line + ci_95_global, color='pink', alpha=0.3)
    
    # 指标标注
    plt.text(0.05, 0.95, f'$R^2={r2:.3f}$\nRMSE={rmse:.3f}\nMAPE={mape:.1f}%',  # 新增MAPE
             transform=plt.gca().transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), fontsize=9)
    
    # 色条（仅最后一个子图）
    if idx == len(results) - 1:
        cbar = plt.colorbar(sc, ax=plt.gca())
        cbar.set_label('预测值', fontsize=9)
    
    # 设置坐标轴范围
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    
    # 标签和标题
    plt.xlabel('观测值', fontsize=10)
    plt.ylabel('预测值', fontsize=10)
    plt.title(f'{model_name}', fontsize=12)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('all_models_predictions_scatter.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ 所有模型散点图汇总已保存为 'all_models_predictions_scatter.png'")

# 9.2 统计检验可视化
if 't_test_df' in locals() and not t_test_df.empty:
    plt.figure(figsize=(12, 6))
    
    # 提取p值
    comparisons = t_test_df['Comparison'].tolist()
    p_values = t_test_df['p_value_one_sided'].tolist()  # 使用单侧p值
    
    # 创建颜色映射（根据显著性）
    colors = []
    for p in p_values:
        if p < 0.001:
            colors.append('red')
        elif p < 0.01:
            colors.append('orange')
        elif p < 0.05:
            colors.append('yellow')
        else:
            colors.append('gray')
    
    # 绘制p值
    bars = plt.barh(range(len(comparisons)), -np.log10(p_values), color=colors)
    plt.axvline(x=-np.log10(0.05), color='blue', linestyle='--', alpha=0.7, label='p=0.05')
    plt.axvline(x=-np.log10(0.01), color='green', linestyle='--', alpha=0.7, label='p=0.01')
    plt.axvline(x=-np.log10(0.001), color='red', linestyle='--', alpha=0.7, label='p=0.001')
    
    plt.xlabel('-log₁₀(p值)')
    plt.title('统计显著性检验 (-log₁₀(p值)，越高越显著)')
    plt.yticks(range(len(comparisons)), comparisons)
    plt.legend()
    plt.tight_layout()
    plt.savefig('statistical_significance_plot.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ 统计显著性图已保存为 'statistical_significance_plot.png'")

# ============ 10. 保存所有结果 ============
print("\n" + "="*60)
print("保存所有结果")
print("="*60)

output_file = 'statistical_analysis_full_results.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # 工作表1: 性能总结（包含MAE和MAPE）
    summary_export_df = summary_df_sorted.copy()
    summary_export_df = summary_export_df[[
        'Model', 'Test_RMSE', 'Test_MAE', 'Test_MAPE', 'Test_R2',
        'CV_RMSE_mean', 'CV_RMSE_std', 'CV_RMSE_CI_lower', 'CV_RMSE_CI_upper'
    ]]
    summary_export_df.to_excel(writer, sheet_name='Performance_Summary', index=False)
    
    # 工作表2: 详细交叉验证结果
    cv_details = []
    for name, res in results.items():
        for fold_idx, score in enumerate(res['cv_rmse_scores']):
            cv_details.append({
                'Model': name,
                'Fold': fold_idx + 1,
                'RMSE': score
            })
    pd.DataFrame(cv_details).to_excel(writer, sheet_name='CV_Details', index=False)
    
    # 工作表3: 预测值
    predictions_data = []
    for model_name in results.keys():
        y_pred = results[model_name]['y_pred']
        y_test_vals = results[model_name]['y_test']
        mape = results[model_name]['mape']
        
        for i in range(len(y_pred)):
            error = y_test_vals[i] - y_pred[i]
            absolute_error = abs(error)
            squared_error = error ** 2
            percentage_error = abs(error / (y_test_vals[i] + 1e-10)) * 100 if y_test_vals[i] != 0 else 0
            
            predictions_data.append({
                'Model': model_name,
                'Sample_ID': i,
                'True_Value': y_test_vals[i],
                'Predicted_Value': y_pred[i],
                'Error': error,
                'Absolute_Error': absolute_error,
                'Squared_Error': squared_error,
                'Percentage_Error': percentage_error
            })
    pd.DataFrame(predictions_data).to_excel(writer, sheet_name='Predictions', index=False)
    
    # 工作表4: 统计检验结果
    if 't_test_df' in locals():
        t_test_df.to_excel(writer, sheet_name='Statistical_Tests', index=False)
    
    # 工作表5: 模型参数
    params_data = []
    for name, res in results.items():
        params_data.append({
            'Model': name,
            'Best_Parameters': str(res['best_params']),
            'Test_RMSE': res['rmse'],
            'Test_MAE': res['mae'],
            'Test_MAPE': res['mape'],
            'Test_R2': res['r2']
        })
    pd.DataFrame(params_data).to_excel(writer, sheet_name='Parameters', index=False)
    
    # 工作表6: VIF分析结果
    if 'vif_results' in locals():
        vif_results.to_excel(writer, sheet_name='VIF_Analysis', index=False)
    
    # 新增工作表7: 汇总统计
    summary_stats = []
    for name, res in results.items():
        summary_stats.append({
            'Model': name,
            'RMSE': res['rmse'],
            'MAE': res['mae'],
            'MAPE': res['mape'],
            'R2': res['r2'],
            'CV_RMSE_Mean': np.mean(res['cv_rmse_scores']),
            'CV_RMSE_Std': np.std(res['cv_rmse_scores'])
        })
    pd.DataFrame(summary_stats).to_excel(writer, sheet_name='Summary_Stats', index=False)

print(f"✓ 所有结果已保存到 '{output_file}'")

# ============ 11. 生成统计报告 ============
print("\n" + "="*60)
print("统计分析报告")
print("="*60)

# 生成文本报告
report_lines = []
report_lines.append("="*80)
report_lines.append("                    模型比较统计报告")
report_lines.append("="*80)
report_lines.append(f"数据集: {df.shape[0]} 样本, {df.shape[1]} 特征")
report_lines.append(f"目标变量: {target_col}")
report_lines.append(f"比较模型数: {len(results)}")
report_lines.append(f"评估指标: RMSE, MAE, MAPE, R²")
report_lines.append(f"置信区间: 使用Bootstrap方法计算 (n=1000)")
report_lines.append("")

# 最佳模型信息
if len(results) > 0:
    best_model_info = summary_df_sorted.iloc[0]
    report_lines.append("🏆 最佳模型性能总结:")
    report_lines.append("-" * 80)
    report_lines.append(f"  模型: {best_model_info['Model']}")
    report_lines.append(f"  测试集RMSE: {best_model_info['Test_RMSE']}")
    report_lines.append(f"  测试集MAE: {best_model_info['Test_MAE']}")
    report_lines.append(f"  测试集MAPE: {best_model_info['Test_MAPE']}")
    report_lines.append(f"  测试集R²: {best_model_info['Test_R2']}")
    report_lines.append(f"  交叉验证RMSE: {best_model_info['CV_RMSE_mean']:.4f} "
                       f"[{best_model_info['CV_RMSE_CI_lower']:.4f}, "
                       f"{best_model_info['CV_RMSE_CI_upper']:.4f}]")
    report_lines.append("")

# 多重共线性总结
if 'vif_results' in locals():
    high_vif = len(vif_results[vif_results["VIF"] >= 10])
    moderate_vif = len(vif_results[(vif_results["VIF"] >= 5) & (vif_results["VIF"] < 10)])
    
    report_lines.append("📊 多重共线性分析:")
    report_lines.append(f"  严重共线性特征 (VIF ≥ 10): {high_vif}")
    report_lines.append(f"  中度共线性特征 (5 ≤ VIF < 10): {moderate_vif}")
    
    if high_vif > 0:
        report_lines.append("  ⚠️  建议检查高VIF特征:")
        for _, row in vif_results[vif_results["VIF"] >= 10].head(3).iterrows():
            report_lines.append(f"    - {row['Feature']}: VIF = {row['VIF']:.2f}")
    report_lines.append("")

# 统计显著性总结
if 't_test_df' in locals():
    sig_count_005 = sum(t_test_df['significant_0.05'])
    sig_count_001 = sum(t_test_df['significant_0.01'])
    sig_count_0001 = sum(t_test_df['significant_0.001'])
    
    report_lines.append("📈 统计显著性检验:")
    report_lines.append(f"  配对t检验显著数量 (p < 0.05): {sig_count_005}/{len(t_test_df)}")
    report_lines.append(f"  配对t检验显著数量 (p < 0.01): {sig_count_001}/{len(t_test_df)}")
    report_lines.append(f"  配对t检验显著数量 (p < 0.001): {sig_count_0001}/{len(t_test_df)}")
    
    if sig_count_005 > 0:
        report_lines.append("  显著优于基准的模型:")
        for _, row in t_test_df[t_test_df['significant_0.05']].iterrows():
            stars = "***" if row['significant_0.001'] else "**" if row['significant_0.01'] else "*" if row['significant_0.05'] else ""
            report_lines.append(f"    - {row['Comparison']} {stars} (d={row['cohens_d']:.3f})")

# 保存报告
report_text = "\n".join(report_lines)
with open('statistical_analysis_report.txt', 'w', encoding='utf-8') as f:
    f.write(report_text)

print(report_text)
print(f"\n✓ 分析报告已保存到 'statistical_analysis_report.txt'")

print("\n" + "="*60)
print("✅ 统计分析完成！")
print("="*60)
print("生成的文件:")
print(f"  1. {output_file} - 所有详细结果")
print(f"  2. vif_analysis_results.xlsx - 多重共线性分析")
print(f"  3. model_comparison_with_ci.png - 模型对比图（包含RMSE/MAE/MAPE）")
print(f"  4. statistical_significance_plot.png - 统计显著性图")
print(f"  5. statistical_analysis_report.txt - 分析报告")
print(f"  6. all_models_predictions_scatter.png - 所有模型散点图汇总")
for model_name in results.keys():
    print(f"  7. {model_name}_predictions_scatter.png - {model_name}散点图（包含MAPE）")
if os.path.exists('vif_analysis.png'):
    print(f"  {8 + len(results) if len(results) > 0 else 7}. vif_analysis.png - VIF可视化图")
print("="*60)

加载数据
✓ 成功加载数据: tabpfn_dataset_1000.xlsx
数据形状: (1000, 15)
列名: ['age', 'experience_level', 'learning_style', 'category', 'difficulty_level', 'ratings', 'num_reviews', 'time_spent_on_course', 'completion_status', 'video_topic', 'video_duration', 'time_watched', 'skip_count', 'pause_count', 'engagement_score']

多重共线性检验 (VIF分析)
计算方差膨胀因子(VIF)...

📊 VIF分析结果:
--------------------------------------------------------------------------------
             Feature      VIF       Collinearity_Level Collinearity_Concern
        time_watched 3.231421 Acceptable (1 < VIF < 5)                  LOW
      video_duration 2.570890 Acceptable (1 < VIF < 5)                  LOW
   completion_status 1.839622 Acceptable (1 < VIF < 5)                  LOW
         num_reviews 1.431341 Acceptable (1 < VIF < 5)                  LOW
             ratings 1.360485 Acceptable (1 < VIF < 5)                  LOW
          skip_count 1.227318 Acceptable (1 < VIF < 5)                  LOW
time_spent_on_course 1.122641 Acc